<a id='top'></a>
# Investigating No-Show Appointments Dataset.

## Introduction
This dataset collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment.

I will be analyizing this dataset to explore What factors are important for us to know in order to predict if a patient will show up for their scheduled appointment?

In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# checking for the csv file name
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/noshowappointments/KaggleV2-May-2016.csv


<a id='wrangling'></a>
## Data Wrangling

#### Inspecting and Assessing the data

In [3]:
# loading the data into a dataframe
original_df = pd.read_csv(os.path.join(dirname, filename))
original_df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [4]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


## what needs to be cleaned:

- Change the Dtype of PatientId and AppointmentId columns to string.
- Change the Dtype of Scholarship, Hipertension, Diabetes, Alcoholism, Handcap, and SMS_recieved columns to bool.
- Extract the day date from ScheduledDay and AppointmentDay columns.
- Change the Dtype of ScheduledDay and AppointmentDay columns to datetime.
- Switch the No-show column to ShowedUp as it's much easier to interpret if no means didn't show up.

### Data Cleaning

In [5]:
# creating a copy to work on.
df_copy = original_df.copy()

- #### Changing columns to the appropriate data types.

In [6]:
df_copy['PatientId'] = df_copy['PatientId'].astype('str')
df_copy['PatientId']

0          29872499824296.0
1         558997776694438.0
2           4262962299951.0
3            867951213174.0
4           8841186448183.0
                ...        
110522      2572134369293.0
110523      3596266328735.0
110524     15576631729893.0
110525     92134931435557.0
110526    377511518121127.0
Name: PatientId, Length: 110527, dtype: object

In [7]:
# removing the decimal point from the patient id
df_copy['PatientId'] = df_copy['PatientId'].str.split('.', expand=True)[0]
df_copy['PatientId']

0          29872499824296
1         558997776694438
2           4262962299951
3            867951213174
4           8841186448183
               ...       
110522      2572134369293
110523      3596266328735
110524     15576631729893
110525     92134931435557
110526    377511518121127
Name: PatientId, Length: 110527, dtype: object

In [8]:
df_copy['AppointmentID'] = df_copy['AppointmentID'].astype('str')
df_copy['AppointmentID']

0         5642903
1         5642503
2         5642549
3         5642828
4         5642494
           ...   
110522    5651768
110523    5650093
110524    5630692
110525    5630323
110526    5629448
Name: AppointmentID, Length: 110527, dtype: object

In [9]:
df_copy['Scholarship'] = df_copy['Scholarship'].astype('bool')
df_copy['Hipertension'] = df_copy['Hipertension'].astype('bool')
df_copy['Diabetes'] = df_copy['Diabetes'].astype('bool')
df_copy['Alcoholism'] = df_copy['Alcoholism'].astype('bool')
df_copy['Handcap'] = df_copy['Handcap'].astype('bool')
df_copy['SMS_received'] = df_copy['SMS_received'].astype('bool')

In [10]:
# extracting the day date from ScheduledDay and AppointmentDay columns.
df_copy['AppointmentDay'] = df_copy['AppointmentDay'].str.split('T', expand=True)[0]
df_copy['ScheduledDay'] = df_copy['ScheduledDay'].str.split('T', expand=True)[0]

In [11]:
# changing to datetime data type.
df_copy['AppointmentDay'] = pd.to_datetime(df_copy['AppointmentDay'])
df_copy['ScheduledDay'] = pd.to_datetime(df_copy['ScheduledDay'])

- #### Switching the No-show column to ShowedUp

In [12]:
df_copy.rename(columns={'No-show': 'ShowedUp'}, inplace=True)

In [13]:
# function to Switch the column values.
def convert(x):
    if x == 'Yes':
        return 'No'
    else:
        return 'Yes'

In [14]:
df_copy['ShowedUp'] = df_copy['ShowedUp'].apply(convert)

In [15]:
df_copy.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,ShowedUp
0,29872499824296,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,False,True,False,False,False,False,Yes
1,558997776694438,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,False,False,False,False,False,Yes
2,4262962299951,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,False,False,False,False,False,False,Yes
3,867951213174,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,False,False,False,False,False,False,Yes
4,8841186448183,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,False,True,True,False,False,False,Yes


In [16]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   PatientId       110527 non-null  object        
 1   AppointmentID   110527 non-null  object        
 2   Gender          110527 non-null  object        
 3   ScheduledDay    110527 non-null  datetime64[ns]
 4   AppointmentDay  110527 non-null  datetime64[ns]
 5   Age             110527 non-null  int64         
 6   Neighbourhood   110527 non-null  object        
 7   Scholarship     110527 non-null  bool          
 8   Hipertension    110527 non-null  bool          
 9   Diabetes        110527 non-null  bool          
 10  Alcoholism      110527 non-null  bool          
 11  Handcap         110527 non-null  bool          
 12  SMS_received    110527 non-null  bool          
 13  ShowedUp        110527 non-null  object        
dtypes: bool(6), datetime64[ns](2), int64

<a id='eda'></a>
## Exploratory Data Analysis

In [17]:
# checking if a single patient made multiple appointments.
df_copy['PatientId'].duplicated().sum()

48228